<a href="https://colab.research.google.com/github/yotamgardosh/LLM-Research-Technion/blob/main/lm_harness_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git@big-refactor
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install llama-cpp-python
!pip install langchain
!pip install langchain-community
!pip install langchain llama-cpp-python
!pip install xlsxwriter
!pip install openpyxl

  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git (to revision big-refactor) to /tmp/pip-req-build-xfkswbsx
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git /tmp/pip-req-build-xfkswbsx
  Running command git checkout -b big-refactor --track origin/big-refactor
  Switched to a new branch 'big-refactor'
  Branch 'big-refactor' set up to track remote branch 'big-refactor' from 'origin'.
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit 967eb4fa90b80ba4e8cc7a2fd171f44f0e384808
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from datasets import load_dataset
from huggingface_hub import notebook_login
from openpyxl import load_workbook
from openpyxl.formatting.rule import CellIsRule
from openpyxl.styles import PatternFill
import pandas as pd
import os
from langchain import LLMChain, PromptTemplate
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
!rm -rf /content/LLM-Research-Technion
!git clone https://github.com/yotamgardosh/LLM-Research-Technion.git
os.chdir('LLM-Research-Technion')
from models_class import Model
from huggingface_hub import notebook_login

notebook_login()

Cloning into 'LLM-Research-Technion'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 86 (delta 35), reused 35 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (86/86), 8.94 MiB | 19.47 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [9]:
mistral_7b = Model("TheBloke/OpenHermes-2.5-Mistral-7B-GGUF","Mistral_7b","openhermes-2.5-mistral-7b.Q4_K_M.gguf")
orca_2_7b = Model("TheBloke/Orca-2-7B-GGUF","Orca2_7b","orca-2-7b.Q4_K_M.gguf")
llama_2_7b = Model("TheBloke/Llama-2-7B-GGUF","Llama2_7b","llama-2-7b.Q4_K_M.gguf")

------------------------------------------------------------------------
Initialized model 'Mistral_7b' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
------------------------------------------------------------------------



------------------------------------------------------------------------
Initialized model 'Orca2_7b' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
--------------

In [10]:
ds = load_dataset("rabell/SysEngBench")
mistral_path = mistral_7b.create_model_path()
orca_path = orca_2_7b.create_model_path()
llama_path = llama_2_7b.create_model_path()



openhermes-2.5-mistral-7b.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

orca-2-7b.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

llama-2-7b.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

In [3]:
YAML_SysEngBench_string = '''
task: demo_sysengbench_zero_shot
dataset_path: rabell/SysEngBench
dataset_name: default  # Replace with specific subset name if applicable
description: "The following are multiple-choice questions (with answers) related to System Engineering."
test_split: test
output_type: multiple_choice
doc_to_text: |
  Question: {{question.strip()}}
  A. {{choiceA}}
  B. {{choiceB}}
  C. {{choiceC}}
  D. {{choiceD}}
  Answer:
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: answer
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
'''

with open('mmlu_SysEngBench.yaml', 'w') as f:
    f.write(YAML_SysEngBench_string)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.0-cp310-cp310-linux_x86_64.whl size=3486852 sha256=79538b7af69ade35d3a73719a62a8bf8073be758e760f87de881b937cf48dde2
  Stored in directory: /root/.cache/pip/wheels/81/72/ce/21e5bb4cc7c4a6799b225426e01bec5eb210e2457eb66321be
Successfully built llama-cpp-python


In [ ]:
!lm_eval \
    --model llamacpp \
    --model_args model_path=/content/Orca-2-7B-GGUF/orca-2-7b.Q4_K_M.gguf,n_ctx=2048,n_threads=4 \
    --include_path ./ \
    --tasks demo_sysengbench_zero_shot \
    --limit 100 \
    --output output/demo_sysengbench_zero_shot/ \
    --log_samples


2024-09-25:11:45:02,687 INFO     [utils.py:145] Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-09-25:11:45:02,687 INFO     [utils.py:148] Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-09-25:11:45:02,687 INFO     [utils.py:161] NumExpr defaulting to 16 threads.
2024-09-25:11:45:02,861 INFO     [config.py:54] PyTorch version 2.4.0+cpu available.
2024-09-25:11:45:02,861 INFO     [config.py:101] TensorFlow version 2.15.0 available.
2024-09-25:11:45:02,862 INFO     [config.py:114] JAX version 0.4.33 available.
2024-09-25 11:45:04.146671: I tensorflow/core/tpu/tpu_api_dlsym_initializer.cc:95] Opening library: /usr/local/lib/python3.10/dist-packages/tensorflow/python/platform/../../libtensorflow_cc.so.2
2024-09-25 11:45:04.146885: I tensorflow/core/tpu/tpu_api_dlsym_initializer.cc:119] Libtpu path is: libtpu.so
2024-09-25 11:45:04.208061: I tensorflow/core/platfor